In [12]:
import pickle
import pvlib
import pandas as pd

In [2]:
result = pickle.load(open("generation.pickle", "rb"))

In [18]:
dataset = pd.DataFrame(result.ac)
dataset.index = pd.to_datetime(dataset.index)
dataset.index.name = "Datetime"

In [21]:
weekly_episodes = dataset.groupby(pd.Grouper(freq='W'))

In [23]:
weekly_episodes.describe()

p_mp                                                       \
            count      mean        std       min   25%   50%       75%   
Datetime                                                                 
2006-01-01   24.0  1.697449  13.621818 -3.030000 -3.03 -3.03 -3.030000   
2006-01-08  168.0  3.207000  15.028463 -3.030000 -3.03 -3.03 -0.439303   
2006-01-15  168.0  1.939853  12.287468 -3.030000 -3.03 -3.03 -3.030000   
2006-01-22  168.0  6.654501  20.149780 -3.030000 -3.03 -3.03  5.686807   
2006-01-29  168.0  5.393326  19.961731 -3.353673 -3.03 -3.03  2.028187   
...           ...       ...        ...       ...   ...   ...       ...   
2010-12-05  168.0  6.404910  19.440513 -4.010676 -3.03 -3.03 -3.030000   
2010-12-12  168.0  3.315945  15.749604 -3.437605 -3.03 -3.03 -3.030000   
2010-12-19  168.0  2.316614  13.428233 -3.030000 -3.03 -3.03 -3.030000   
2010-12-26  168.0  4.858090  16.694981 -3.531632 -3.03 -3.03 -3.030000   
2011-01-02  120.0  4.884165  17.541947 -3.498425 -3.03 -3.03 -3.030000   

                       
                  max  
Datetime               
2006-01-01  51.639288  
2006-01-08  68.179388  
2006-01-15  65.029365  
2006-01-22  79.884108  
2006-01-29  94.812569  
...               ...  
2010-12-05  73.983002  
2010-12-12  69.827375  
2010-12-19  65.142728  
2010-12-26  63.218910  
2011-01-02  67.174872  

[262 rows x 8 columns]

In [25]:
with pd.HDFStore('./datasets/generation.h5') as store:
    # Save each DataFrame with a key
    for i, (name, df) in enumerate(weekly_episodes):
        store[f'eps_{i}'] = df

In [1]:
import pandas as pd

In [2]:
test = pd.read_hdf('./datasets/generation.h5', key='eps_0')

In [3]:
test

,p_mp
Datetime,
2006-01-01 00:10:00,-3.030000
2006-01-01 01:10:00,-3.030000
2006-01-01 02:10:00,-3.030000
2006-01-01 03:10:00,-3.030000
2006-01-01 04:10:00,-3.030000
2006-01-01 05:10:00,-3.030000
2006-01-01 06:10:00,-3.030000
2006-01-01 07:10:00,-3.030000
2006-01-01 08:10:00,-3.030000


In [4]:
from datetime import datetime

In [21]:
test

,p_mp
Datetime,
2006-01-01 00:10:00,-3.030000
2006-01-01 01:10:00,-3.030000
2006-01-01 02:10:00,-3.030000
2006-01-01 03:10:00,-3.030000
2006-01-01 04:10:00,-3.030000
2006-01-01 05:10:00,-3.030000
2006-01-01 06:10:00,-3.030000
2006-01-01 07:10:00,-3.030000
2006-01-01 08:10:00,-3.030000


In [18]:
test.loc[datetime(2006, 1, 1, 9, 10).strftime("%Y-%m-%d %H:%M:%S")]

p_mp    17.752944
Name: 2006-01-01 09:10:00, dtype: float64

In [29]:
test.iloc[test.index.get_indexer([pd.to_datetime(datetime(2006,1,1, 9))], method='nearest')]

,p_mp
Datetime,
2006-01-01 09:10:00,17.752944
